In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

# Pré-traitement

## Chargement des données

In [2]:
data_bus = pd.read_csv("data_estia_bihar_passagers.csv",parse_dates=['dateTime']).sort_values("dateTime")
data_bus.shape

(36901, 4)

In [3]:
data_bus.dtypes

dateTime            datetime64[ns]
passengersNumber           float64
line                        object
lineType                    object
dtype: object

## Vérification des valeurs manquantes et doublons

In [4]:
# Vérification de l'existence de valeurs manquantes
n_missval = data_bus.isnull().any(axis=1).sum()
print(f"Il y'a {n_missval} valeur(s) manquante(s)")

Il y'a 0 valeur(s) manquante(s)


In [5]:
# Vérification de l'existence des doublons
n_dbl = data_bus.duplicated(["dateTime","line","lineType"]).sum()
print(f"Il y'a {n_dbl} doublon(s)")

Il y'a 58 doublon(s)


In [6]:
# Suppression des doublons
data_bus2 = data_bus.drop_duplicates(["dateTime","line","lineType"])
n_dbl = data_bus2.duplicated(["dateTime","line","lineType"]).sum()
print(f"Il y'a {n_dbl} doublons")
print(f"Il y'a {data_bus2.shape[0]} observations après suppression des doublons")

Il y'a 0 doublons
Il y'a 36843 observations après suppression des doublons


## Vérification et correction des types de données

In [7]:
data_bus2.dtypes

dateTime            datetime64[ns]
passengersNumber           float64
line                        object
lineType                    object
dtype: object

## Ajout des données supplémentaires

### Méteo

In [8]:
data_meteo_2019 = pd.read_csv("data/meteo/export-saint-sebastien2019.csv",parse_dates=['DATE'])
data_meteo_2020 = pd.read_csv("data/meteo/export-saint-sebastien2020.csv",parse_dates=['DATE'])
data_meteo_2021 = pd.read_csv("data/meteo/export-saint-sebastien2021.csv",parse_dates=['DATE'])
data_meteo_2022 = pd.read_csv("data/meteo/export-saint-sebastien2022.csv",parse_dates=['DATE'])
data_meteo_2023 = pd.read_csv("data/meteo/export-saint-sebastien2023.csv",parse_dates=['DATE'])

In [9]:
# Concaténation des données méteo
data_meteo_2019_2023 = pd.concat([data_meteo_2019,data_meteo_2020,data_meteo_2021,data_meteo_2022,data_meteo_2023])
x = pd.merge(data_bus2,data_meteo_2019_2023,left_on='dateTime', right_on='DATE',how="left")

x[x.DATE.isnull()].head(5)

Il manque les données méteo du mois de mars 2023. On va utiliser majoré les données sur les données de mars au niveau l'opinion sur le temps qui était "météo correcte"

In [10]:
x.loc[x.DATE.isnull(),"OPINION"] = "météo correcte"

In [11]:
x[x.DATE.isnull()].shape

(273, 29)

On va conserver les données avec l'opinion sur la méteo

In [12]:
data_bus3 = x.copy()[["dateTime","passengersNumber","line","lineType","OPINION"]]

In [13]:
data_bus3.rename({
    "OPINION":"meteo_status"
},inplace=True,axis=1)

In [14]:
data_bus3

,dateTime,passengersNumber,line,lineType,meteo_status
0,2019-04-05,418.0,42,daily,météo défavorable
1,2019-04-05,1000.0,16,daily,météo défavorable
2,2019-04-05,113.0,B6,night,météo défavorable
3,2019-04-05,3032.0,29,daily,météo défavorable
4,2019-04-05,4016.0,25,daily,météo défavorable
...,...,...,...,...,...
36838,2023-03-08,2401.0,19,daily,météo correcte
36839,2023-03-08,1162.0,41,daily,météo correcte
36840,2023-03-08,1445.0,21,daily,météo correcte
36841,2023-03-08,905.0,23,daily,météo correcte


In [15]:
data_bus3.to_csv("data_processed/data_w_meteo.csv",index=False)